AUTOENCODER-BASED NOVELTY SEARCH

In [47]:
import numpy as np
import random
import math
from itertools import permutations
from google.colab import files
import zipfile
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import shutil
from google.colab import files
import glob
import numpy as np
from sklearn.model_selection import train_test_split

Set seed for reproducibility

In [48]:
import torch

def set_global_seed(seed=42):
    random.seed(seed)

    np.random.seed(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # Para múltiples GPUs

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)
    
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

set_global_seed(42)


Load instances

In [49]:
lolib = np.load('/kaggle/input/benchmark-instances/LOLIB_instances.npy')
print(lolib.shape)
random_instances = np.load('/kaggle/input/benchmark-instances/random_instances.npy')
print(random_instances.shape)
all_instances = np.load('/kaggle/input/benchmark-instances/benchmark_instances.npy')
print(all_instances.shape)
train_3, test_3 = train_test_split(all_instances, test_size=0.2, random_state=42)
train = train_3
validation = test_3
test = test_3
set_global_seed(42)
print(train.shape)

(3188, 20, 20)
(1500, 20, 20)
(4688, 20, 20)
(3750, 20, 20)


Define and train the autoencoder

In [50]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim, hidden_dim=80):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, latent_dim),
            nn.Tanh()
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, input_dim),
            nn.Tanh()
        )

    def forward(self, x):
        z = self.encoder(x)
        out = self.decoder(z)
        return out

def ae_loss(recon_x, x):
    return F.mse_loss(recon_x, x, reduction='mean')

def train_autoencoder(model, data, epochs=50, batch_size=64, lr=1e-3):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for epoch in range(epochs):
        model.train()
        for i in range(0, len(data), batch_size):
            batch = data[i:i+batch_size]
            optimizer.zero_grad()
            recon = model(batch)
            loss = ae_loss(recon, batch)
            loss.backward()
            optimizer.step()

def get_ae_latents(model, data, batch_size=64):
    model.eval()
    latents = []
    with torch.no_grad():
        for i in range(0, len(data), batch_size):
            batch = data[i:i+batch_size]
            z = model.encoder(batch)
            latents.append(z)
    return torch.cat(latents, dim=0)

def train_autoencoder_from_matrices(matrix_data, N, latent_dim=12, hidden_dim=80, epochs=15, batch_size=32, lr=1e-3):
    matrix_array = np.array(matrix_data, dtype=np.float32)

    # Mask to remove diagonal
    mask = ~np.eye(N, dtype=bool)
    masked_data = matrix_array[:, mask]

    # Flatten each masked matrix into a vector
    flat_data = masked_data.reshape(masked_data.shape[0], -1)

    # Convert to torch tensor
    tensor_data = torch.tensor(flat_data, dtype=torch.float32)

    # Define and train the model
    input_dim = tensor_data.shape[1]
    ae_model = Autoencoder(input_dim=input_dim, latent_dim=latent_dim, hidden_dim=hidden_dim)
    train_autoencoder(ae_model, tensor_data, epochs=epochs, batch_size=batch_size, lr=lr)

    # Get latent representation
    latents = get_ae_latents(ae_model, tensor_data, batch_size=batch_size)

    return latents, ae_model

In [51]:
set_global_seed(42)
z, ae_model = train_autoencoder_from_matrices(train, 20)
print(np.max(z.numpy()))
print(np.mean(z.numpy()))

0.94285303
-0.08491668


Define LS algorithms

In [52]:
import numpy as np
from numba import njit

@njit
def profit_permutation(A, sigma):
    total = 0.0
    N = len(sigma)
    for i in range(N):
        for j in range(i + 1, N):
            total += A[sigma[i], sigma[j]]
    return total

In [53]:
@njit
def two_opt_ls(A, rand_indices):
    N = A.shape[0]
    sigma = np.arange(N)
    for i in range(N-1, 0, -1):
        j = rand_indices[N - 1 - i]
        sigma[i], sigma[j] = sigma[j], sigma[i]

    a = profit_permutation(A, sigma)
    improvement = True
    while improvement:
        improvement = False
        for i in range(N):
            for j in range(i+1, N):
                sigma[i:j+1] = sigma[i:j+1][::-1]
                b = profit_permutation(A, sigma)
                if b > a:
                    a = b
                    improvement = True
                    break
                else:
                    sigma[i:j+1] = sigma[i:j+1][::-1]
            if improvement:
                break

    return sigma

In [54]:
@njit
def swap_ls(A, rand_indices):
    N = A.shape[0]
    sigma = np.arange(N)
    for i in range(N - 1, 0, -1):
        j = rand_indices[N - 1 - i]
        sigma[i], sigma[j] = sigma[j], sigma[i]

    current_profit = profit_permutation(A, sigma)
    improvement = True
    while improvement:
        improvement = False
        for i in range(N):
            for j in range(i + 1, N):
                sigma[i], sigma[j] = sigma[j], sigma[i]
                new_profit = profit_permutation(A, sigma)

                if new_profit > current_profit:
                    current_profit = new_profit
                    improvement = True
                    break
                else:
                    sigma[i], sigma[j] = sigma[j], sigma[i]
            if improvement:
                break

    return sigma

In [55]:
@njit
def insert_ls(A, rand_indices):
    N = A.shape[0]
    sigma = np.arange(N)
    for i in range(N-1, 0, -1):
        j = rand_indices[N - 1 - i]
        sigma[i], sigma[j] = sigma[j], sigma[i]

    a = profit_permutation(A, sigma)
    improvement = True
    while improvement:
        improvement = False
        for i in range(N):
            for j in range(N):
                if i == j:
                    continue
                sigma_copy = sigma.copy()
                temp = sigma_copy[i]
                if i < j:
                    for k in range(i, j):
                        sigma_copy[k] = sigma_copy[k+1]
                else:
                    for k in range(i, j, -1):
                        sigma_copy[k] = sigma_copy[k-1]
                sigma_copy[j] = temp

                b = profit_permutation(A, sigma_copy)
                if b > a:
                    sigma = sigma_copy
                    a = b
                    improvement = True
                    break
            if improvement:
                break

    return sigma

Define the NS algorithm and all the necessary functions

In [56]:
def profit(A, algorithm):
    rand_indices = [np.random.randint(0, i+1) for i in range(N-1, 0, -1)]
    sigma = np.array(algorithm(A, rand_indices))
    idx = sigma.astype(int)

    A_sub = A[np.ix_(idx, idx)]
    total_sum = np.sum(np.triu(A_sub, k=1))
    return total_sum

In [57]:
def initialise(D, N):
    # D[0] = population size, D[1] = values for uniform distribution (-D[1], D[1])
    num_all_instances = D[0]
    max_valor = D[1]

    population = np.empty((num_all_instances, N, N))

    for idx in range(num_all_instances):
        A = np.random.uniform(-max_valor, max_valor, size=(N, N))
        np.fill_diagonal(A, 0)
        population[idx] = A
    return population


In [58]:
def profit_portfolio(A, portfolio):
    f = np.array([profit(A, algo) for algo in portfolio])
    return f.reshape(-1, 1)

In [59]:
import numpy as np
import math

def novelty_score(population, archive, k):
    pop_size = len(population)
    archive_size = len(archive)
    N = population[0].shape[0]

    U = np.zeros((archive_size, 12))
    D = np.zeros((pop_size, 12))

    lower_indices = np.tril_indices(N, k=-1)

    mask = ~np.eye(N, dtype=bool)
    masked_data = archive[:,mask]
    flat = masked_data.reshape(masked_data.shape[0], -1)
    tens = torch.tensor(np.array(flat), dtype=torch.float32)
    ae_model.eval()
    with torch.no_grad():
        U = ae_model.encoder(tens).cpu().numpy().astype(np.float32)

    mask = ~np.eye(N, dtype=bool)
    masked_data = population[:,mask]
    flat = masked_data.reshape(masked_data.shape[0], -1)
    tens = torch.tensor(np.array(flat), dtype=torch.float32)
    ae_model.eval()
    with torch.no_grad():
        D = ae_model.encoder(tens).cpu().numpy().astype(np.float32)

    all_descriptors = np.concatenate((U, D), axis=0)
    total_descriptors = all_descriptors.shape[0]

    scores = np.zeros((pop_size, 1))
    for t in range(pop_size):
        distances = np.linalg.norm(all_descriptors - D[t, :], axis=1)
        self_index = archive_size + t
        distances[self_index] = np.inf
        sorted_dist = np.sort(distances)
        finite_dists = sorted_dist[np.isfinite(sorted_dist)]
        k_eff = min(k, len(finite_dists))
        scores[t] = np.mean(finite_dists[:k_eff])

    return scores

In [60]:
def performance_score(population, portfolio, R=10):
    amount_algorithms = len(portfolio)
    pop_size = population.shape[0]

    performance_scores = np.zeros((pop_size, amount_algorithms))

    for i in range(pop_size):
        cumulative_profit = np.zeros(amount_algorithms)

        for r in range(R):
            profit_vec = profit_portfolio(population[i], portfolio)
            profit_vec = profit_vec.flatten()
            cumulative_profit += profit_vec

        performance_scores[i, :] = cumulative_profit / R

    for i in range(pop_size):
        target_value = performance_scores[i, 0]
        other_values = performance_scores[i, 1:]
        performance_scores[i, 0] = target_value - np.max(other_values)

    return performance_scores

In [61]:
def evaluate(population, archive, portfolio, k, phi, R=10):
    pop_size = population.shape[0]

    novelty = novelty_score(population, archive, k)
    performance = performance_score(population, portfolio, R)

    novelty = np.asarray(novelty).flatten()

    target_performance = np.asarray(performance[:, 0]).flatten()

    novelty_std = (novelty - np.mean(novelty)) / np.std(novelty)
    target_performance_std = (target_performance - np.mean(target_performance)) / np.std(target_performance)

    fitness = phi * target_performance_std + (1 - phi) * novelty_std

    return fitness.reshape(-1, 1)

In [62]:
def cross(instance_1, instance_2, N):
    mask = ~np.eye(N, dtype=bool)
    flat_1 = instance_1[mask].flatten()
    flat_2 = instance_2[mask].flatten()

    dimension = N * N - N
    crossover_mask = np.random.randint(0, 2, dimension)

    offspring1 = np.where(crossover_mask, flat_1, flat_2)
    offspring2 = np.where(crossover_mask, flat_2, flat_1)

    return offspring1, offspring2

In [63]:
def mutation(instance, mutation_rate=0.01):
    instance = instance.copy()
    mask = np.random.rand(len(instance)) < mutation_rate
    instance[mask] = np.random.uniform(-1, 1, np.sum(mask))
    return instance

In [64]:
def array_to_matrix(array, N):
    matrix = np.zeros((N, N))
    idx = 0
    for i in range(N):
        for j in range(N):
            if i != j:
                matrix[i, j] = array[idx]
                idx += 1
    return matrix

In [65]:
def offspring(population, archive, portfolio, k, phi, mutation_rate=0.01, R=10):
    pop_size, N, _ = population.shape
    flat_size = N * N - N
    
    fitness_values = evaluate(population, archive, portfolio, k, phi, R)
    indexes = np.arange(pop_size)

    parents = []
    for _ in range(2 * pop_size):
        a, b = random.choices(indexes, k=2)
        winner = a if fitness_values[a] > fitness_values[b] else b  # Cambia a < si es minimización
        parents.append(winner)
    parents = np.array(parents).reshape(pop_size, 2)

    offspring_flat = np.zeros((pop_size, flat_size))
    for i in range(0, pop_size, 2):
        p1 = population[parents[i, 0]]
        p2 = population[parents[i, 1]]
        o1, o2 = cross(p1, p2, N)
        o1 = mutation(o1, mutation_rate)
        o2 = mutation(o2, mutation_rate)
        offspring_flat[i] = o1
        offspring_flat[i + 1] = o2

    offspring_matrix = np.array([array_to_matrix(vec, N) for vec in offspring_flat])
    return offspring_matrix

In [66]:
def update_archive(population, archive, portfolio, k, phi, ta, R=10):
    pop_size = len(population)
    pop_novelty = novelty_score(population, archive, k)
    pop_perf = performance_score(population, portfolio, R)

    for i in range(pop_size):
        if random.random() < 0.01:
            archive = np.append(archive, [population[i]], axis=0)
        elif pop_perf[i, 0] > 0 and pop_novelty[i] > ta:
            archive = np.append(archive, [population[i]], axis=0)

    return archive

In [67]:
def update_ss(population, solution_set, portfolio, phi, tss, R=10):
    pop_size = len(population)
    pop_perf = performance_score(population, portfolio, R)

    for i in range(pop_size):
        if pop_perf[i, 0] > 0:
            novelty = novelty_score(np.array([population[i]]), solution_set, k=1)[0]
            if novelty > tss:
                solution_set = np.append(solution_set, [population[i]], axis=0)

    return solution_set

In [68]:
def novelty_search(D, N, k, phi, generations, portfolio, ta, tss, R=10, mutation_rate=0.01, initial_archive = [], population = []):
    if len(population) == 0:
        population = initialise(D, N)
    else:
        population = population

    if len(initial_archive) == 0:
        archive = np.array([random.choice(population)])
    else:
        archive = initial_archive

    solution_set = archive
    archive = update_archive(population, archive, portfolio, k, phi, ta, R)
    solution_set = update_ss(population, solution_set, portfolio, phi, tss, R)

    for i in range(generations):
        offspring_pop = offspring(population, archive, portfolio, k, phi, mutation_rate, R)

        # Evaluar población original y offspring
        combined = np.concatenate((population, offspring_pop), axis=0)
        fitness = evaluate(combined, archive, portfolio, k, phi, R).flatten()

        # Dividir fitness en mitades
        fitness_parents = fitness[:len(population)]
        fitness_offspring = fitness[len(population):]

        # Selección entre padre e hijo por posición
        new_population = []
        for j in range(len(population)):
            if fitness_offspring[j] > fitness_parents[j]:
                new_population.append(offspring_pop[j])
            else:
                new_population.append(population[j])
                
        population = np.array(new_population)

        archive = update_archive(population, archive, portfolio, k, phi, ta, R)
        solution_set = update_ss(population, solution_set, portfolio, phi, tss, R)
        
        if (i+1) in [250, 500, 750, 1000]:
            name = portfolio[0].py_func.__name__
            np.save(f"{name}_{i+1}_gens_NS2_0_4.npy", solution_set)
            print(f"[Checkpoint] Saved in generation {i+1}")

    return solution_set

Run the algorithm and save the results for 250, 500, 750 and 1000 generations

In [69]:
import numpy as np
from sklearn.model_selection import train_test_split

set_global_seed(42)

D = [50, 1]
N = 20
k = 3
phi = 0.85
generations = 1000
ta = 0.3
ts = 0.3
R=3

training_set, population_set = train_test_split(train,test_size=D[0] / len(train),random_state=42)

print(training_set.shape)
print(population_set.shape)

portfolio_1 = np.array([insert_ls, two_opt_ls, swap_ls])
portfolio_2 = np.array([swap_ls, two_opt_ls, insert_ls])
portfolio_3 = np.array([two_opt_ls, swap_ls, insert_ls])

solution_set_1 = novelty_search(D, N, k, phi, generations, portfolio_1, ta, ts, R, 1 / ((N * N) - N), [], population_set)
solution_set_2 = novelty_search(D, N, k, phi, generations, portfolio_2, ta, ts, R, 1 / ((N * N) - N), [], population_set)
solution_set_3 = novelty_search(D, N, k, phi, generations, portfolio_3, ta, ts, R, 1 / ((N * N) - N), [], population_set)

size_1 = len(solution_set_1)
size_2 = len(solution_set_2)
size_3 = len(solution_set_3)
print(f"Tamaños: {size_1}, {size_2}, {size_3}")

(3700, 20, 20)
(50, 20, 20)
Tamaños: 190, 1, 1
